# AI Based - Flight Control

In [1]:
# SETUP at the Initial stage (run this block one time only)
! pip install pyttsx3 # On linux make sure that 'espeak' and 'ffmpeg' are installed
! pip install pillow

! pip install speechrecognition
# Command for linux users 
#! sudo apt-get install python3-pyaudio # Uncomment This Line in Linux...
# Windows users can install pyaudio
! pip install pyaudio
! pip install pyttsx3

#! pip install textblob
! pip install vaderSentiment
! pip3 install librosa 
! pip install soundfile
! pip install numpy
! pip install sklearn
! pip install pyaudio

! pip install mysql-connector-python

Defaulting to user installation because normal site-packages is not writeable


In [19]:
# Config Part

speechRate = 180
speechVolume = 1.0
speechGender = 1

## Text To Speech Function

In [20]:
import pyttsx3

def tts(text):
    engine = pyttsx3.init()
    voices = engine.getProperty('voices')  
    engine.setProperty('rate', speechRate) 
    engine.setProperty('volume', speechVolume) 
    engine.setProperty('voice', voices[speechGender].id)
    engine.say(text)
    engine.runAndWait()

# to test function
tts("Navy 512, Runway 23 right, winds 15 gusting to 25 from southwest, cleared to land. i repeat, Navy 512, Runway 23 right, winds 15 gusting to 25 from southwest, cleared to land.")

## Speech To Text Function

In [1]:
import speech_recognition as spR
import pyttsx3

reco = spR.Recognizer()

def SpeakText(command):
	
	engine = pyttsx3.init()
	engine.say(command)
	engine.runAndWait()

while(1):
	try:
		with spR.Microphone() as source2:
			reco.adjust_for_ambient_noise(source2, duration=0.2)
			audio2 = reco.listen(source2)
			# Using google to recognize audio
			MyText = reco.recognize_google(audio2)
			#MyText = MyText.lower()

			print("Pilot Says: {0}".format(MyText))
			
	except spR.RequestError as error:
		print("Could not request results; {0}".format(error))
	except spR.UnknownValueError:
		print("unknown error occured")


Pilot Says: Hi I hope it's going great hi hi hi hi hi


KeyboardInterrupt: 

## Pre-Flight Check
A camera will find out the difference by comparing the Given parameter with the current parameters using Computer Vision and if it passes all the criteria then Flight will be ready to fly or else the pilot will be asked to ReCheck sharply again from the beginning

In [21]:
from PIL import Image ,ImageChops

# using OpenCV we will take an image of the DashBoard then we will bring it to this function for analyzing the image
def pfc(img1,img2,see=False):
    diff = ImageChops.difference(img1,img2)
    if(diff.getbbox()):
        if(see):
            diff.show()
            return True
        else:
            return True
    else:
        return False

# Load 2 Image
imgO = Image.open("mainIMG.jpg")
imgT = Image.open("thereisachange.jpg")
isDiff = pfc(imgO,imgT,True)
if(isDiff):
    print("There is some Difference Sported if I compare it with the actual parameters.")
    tts("There are some changes noticed, Pilot please re-check. the differences they are on your screen.")
else:
    print("There is NO Difference Sported.")
    tts("All the parameters are checked and everything looks fine, Pilot is Clear to move forward with other procedures.")

There is some Difference Sported if I compare it with the actual parameters.


# NLP Part
### Lexicon Based methods to see the emotion statements...

In [3]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

def lexicon(sentence):
    sid_obj= SentimentIntensityAnalyzer()
    data = sid_obj.polarity_scores(sentence)
    return data


# For Testing
data = lexicon("hello i am so excited")
print(data)

{'neg': 0.0, 'neu': 0.562, 'pos': 0.438, 'compound': 0.4795}


### Voice Modulation to see the emotion statements...

In [6]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [8]:
#DataFlair - Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [9]:
#DataFlair - Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

#DataFlair - Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [10]:
#DataFlair - Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob(".\\dataSet\\Actor_*\\*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [11]:
#DataFlair - Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

C:\Users\SukarnaJana\AppData\Local\Temp\ipykernel_10264\4098696342.py:16: FutureWarning: Pass y=[0. 0. 0. ... 0. 0. 0.] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
C:\Users\SukarnaJana\AppData\Local\Temp\ipykernel_10264\4098696342.py:16: FutureWarning: Pass y=[0.0000000e+00 3.0517578e-05 3.0517578e-05 ... 0.0000000e+00 0.0000000e+00
 0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
C:\Users\SukarnaJana\AppData\Local\Temp\ipykernel_10264\4098696342.py:16: FutureWarning: Pass y=[ 0.0000000e+00  0.0000000e+00 -3.0517578e-05 ... -3.0517578e-05
 -3.0517578e-05 -3.0517578e-05] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rat

In [13]:
#DataFlair - Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))
#DataFlair - Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

(576, 192)
Features extracted: 180


In [14]:
#DataFlair - Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [15]:
#DataFlair - Train the model
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [16]:
#DataFlair - Predict for the test set
y_pred=model.predict(x_test)

In [18]:
#DataFlair - Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

#DataFlair - Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 57.81%


## ATC to move flights form which runway...

there will be a central DB from where all the RealTime data will be saved and updated... the Central Computer will fetch the data and allow the filght to have a ground clearence to takeoff

In [ ]:
import mysql.connector as db

mydb = db.connect(
  host="localhost",
  user="root",
  password="1234"
)

def seeFlight():
  data = []
  mycursor = mydb.cursor()
  mycursor.execute("SELECT * FROM flightOnGND where FlightStats = \"HOLD\" ORDER BY \"TakeOffTime\"")
  myresult = mycursor.fetchall()
  for i in myresult:
    data.append(i)
  return data

def seeCurrentFlightStats(flightName):
  mycursor = mydb.cursor()
  mycursor.execute("SELECT FlightStats FROM flightOnGND where FlightID = \"{0}\"".format(flightName))
  myresult = mycursor.fetchall()
  if(myresult=="0"):
    return 'h'
  elif(myresult=="1"):
    return 'm'
  elif(myresult=="2"):
    return 't'
  else:
    return 'o'

def isRunwayFree(runwayID):
  mycursor = mydb.cursor()
  mycursor.execute("SELECT FlightStats,FlightID FROM flightOnGND where RundayID = \"{0}\"".format(runwayID))
  myresult = mycursor.fetchall()
  if(seeCurrentFlightStats())